## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium

## Data Read-in

In [2]:
df = pd.read_csv('redfin_2023-07-06-05-23-36.csv')

## Data Clean

In [3]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [4]:
df = df.dropna(subset=['SOLD DATE'])

In [5]:
# Define list of desired months (excluding current month)
desired_months = ['June']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [6]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    822
Name: count, dtype: int64
-------
$/SQUARE FEET
False    816
True       6
Name: count, dtype: int64
-------
YEAR BUILT
False    822
Name: count, dtype: int64
-------


In [7]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [8]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [9]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
353,PAST SALE,June-9-2023,Condo/Co-op,331 Knotty Pine Cir Unit A-1,Greenacres,FL,33463.0,170.0,2.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Greenacres/331-Knott...,Beaches MLS,RX-10869151,N,Y,26.609970,-80.132466
45,PAST SALE,June-7-2023,Condo/Co-op,7786 Lakeside Blvd #676,Boca Raton,FL,33434.0,310.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/7786-Lake...,Beaches MLS,RX-10885824,N,Y,26.378671,-80.167671
13,PAST SALE,June-7-2023,Condo/Co-op,910 E Tiffany Dr #2,Mangonia Park,FL,33407.0,62258.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Mangonia-Park/910-E-...,Beaches MLS,RX-10896522,N,Y,26.757509,-80.083350
6,PAST SALE,June-7-2023,Condo/Co-op,820 W Tiffany Dr #1,Mangonia Park,FL,33407.0,67860.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Mangonia-Park/820-W-...,Beaches MLS,RX-10896505,N,Y,26.757440,-80.084144
24,PAST SALE,June-30-2023,Condo/Co-op,210 E Tiffany Dr #2,Mangonia Park,FL,33407.0,70000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Mangonia-Park/210-E-...,Beaches MLS,RX-10898498,N,Y,26.756008,-80.082324
350,PAST SALE,June-19-2023,Condo/Co-op,215 Camden I,West Palm Beach,FL,33417.0,72000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/215-...,Beaches MLS,RX-10879571,N,Y,26.713934,-80.135534
405,PAST SALE,June-9-2023,Condo/Co-op,328 Andover M,West Palm Beach,FL,33417.0,74000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/328-...,Beaches MLS,RX-10881587,N,Y,26.709237,-80.127727
250,PAST SALE,June-8-2023,Condo/Co-op,"900 E E Tiffany Drive, 4 Dr",West Palm Beach,FL,33407.0,75000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/900-...,Beaches MLS,RX-10876590,N,Y,26.757868,-80.083120
443,PAST SALE,June-16-2023,Condo/Co-op,326 Northampton P,West Palm Beach,FL,33417.0,76000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/326-...,Beaches MLS,RX-10880439,N,Y,26.718060,-80.133588
809,PAST SALE,June-8-2023,Condo/Co-op,816 W Tiffany Dr #1,Mangonia Park,FL,33407.0,78829.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Mangonia-Park/816-W-...,MARMLS,A11260816,N,Y,26.757042,-80.084070


In [15]:
print(df_filtered['URL'].iloc[45])

https://www.redfin.com/FL/Boca-Raton/7786-Lakeside-Blvd-33434/unit-676/home/42247852


In [15]:
# Correct the prices, if needed
df_filtered.at[353,'PRICE']=(170000)
df_filtered.at[45,'PRICE']=(310000)

In [17]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [18]:
# # Corrections, if needed
df_filtered.at[353,'$/SQUARE FEET']=(170000/353)
df_filtered.at[45,'$/SQUARE FEET']=(310000/1050)

In [19]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
13,62258.0,910 E Tiffany Dr #2,Mangonia Park,64.0
668,85000.0,6805 Willow Wood Dr #5012,Boca Raton,68.0
597,115000.0,33 Stratford Ln Unit B,Boynton Beach,70.0
24,70000.0,210 E Tiffany Dr #2,Mangonia Park,71.0
713,135000.0,6 Stratford Dr Unit B,Boynton Beach,83.0
438,136000.0,12 Southport Ln Unit D,Boynton Beach,84.0
6,67860.0,820 W Tiffany Dr #1,Mangonia Park,85.0
250,75000.0,"900 E E Tiffany Drive, 4 Dr",West Palm Beach,89.0
127,95000.0,309 Circle Dr S #205,Boynton Beach,97.0
809,78829.0,816 W Tiffany Dr #1,Mangonia Park,99.0


In [20]:
print(df_filtered.URL.iloc[13])

https://www.redfin.com/FL/Mangonia-Park/910-E-Tiffany-Dr-33407/unit-2/home/42431003


In [17]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [21]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [22]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [23]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [24]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [25]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [26]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"June 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map

In [27]:
m.save('index.html')

## Summary Info

In [28]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [29]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [30]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [31]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['URL'])

https://www.redfin.com/FL/West-Palm-Beach/200-Arkona-Ct-33401/unit-804/home/185752932


In [32]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
4001 North Ocean Condo, 4001 N Ocean Blvd #504 Gulf Stream | Price $8,000,000 | $2,449 psf | Year built: 2013
Least Expensive
Pine Ridge South Iii Condo, 331 Knotty Pine Cir Unit A-1 Greenacres | Price $170 | $482 psf | Year built: 1981
Highest Price Per Square Foot
4001 North Ocean Condo, 4001 N Ocean Blvd #504 Gulf Stream | Price $8,000,000 | $2,449 psf | Year built: 2013
Lowest Price Per Square Foot
Tiffany Lake Condo PH 1, 910 E Tiffany Dr #2 Mangonia Park | Price $62,258 | $64 psf | Year built: 1973
Newest
La Clara, 200 Arkona Ct #804 West Palm Beach | Price $3,979,920 | $1,286 psf | Year built: 2023
Oldest
Palm Beach Hotel Condo, 235 Sunrise Ave #3201 Palm Beach | Price $550,000 | $1,146 psf | Year built: 1925


## Time on Market Calculator

In [29]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [42]:
from datetime import datetime, timedelta

date1 = datetime(2023, 3, 1) ## List (Earlier) date
date2 = datetime(2023, 6, 29) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

120


## Map URL Snagger

In [39]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [40]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MD_condo_sales_month_ending_june_2023


## Get Summary Data

In [38]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 968
--------
Total sale price: $699,661,349
--------
Median sale price: $400,000
--------
Max sale price: $19,750,000
--------
Min sale price: $72,500
------------------------------------------------
PSF INFO
Max price per square foot: $3,453
--------
Min price per square foot: $82
--------
Median price per square foot: $433
------------------------------------------------
CONDO AGES
Newest building: 2022.0
----------
Oldest building: 1923.0
----------
Average building age: 1988.3543388429753
